In [35]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics





In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SmokeDetction") \
    .getOrCreate()

In [26]:
df=spark.read.csv("./smoke_detection_iot.csv",inferSchema=True,header=True)

In [27]:
df.columns

['_c0',
 'UTC',
 'Temperature[C]',
 'Humidity[%]',
 'TVOC[ppb]',
 'eCO2[ppm]',
 'Raw H2',
 'Raw Ethanol',
 'Pressure[hPa]',
 'PM1',
 'PM2',
 'NC0',
 'NC1',
 'NC2',
 'CNT',
 'Fire Alarm']

In [16]:
df.drop('UTC')

y = df.select(col('Fire Alarm'))
y.describe().show()

df1=df
X=df1.drop(col("Fire Alarm")) \
  .printSchema()


+-------+-------------------+
|summary|         Fire Alarm|
+-------+-------------------+
|  count|              62630|
|   mean| 0.7146255787961041|
| stddev|0.45159618818066594|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

root
 |-- _c0: string (nullable = true)
 |-- UTC: string (nullable = true)
 |-- Temperature[C]: string (nullable = true)
 |-- Humidity[%]: string (nullable = true)
 |-- TVOC[ppb]: string (nullable = true)
 |-- eCO2[ppm]: string (nullable = true)
 |-- Raw H2: string (nullable = true)
 |-- Raw Ethanol: string (nullable = true)
 |-- Pressure[hPa]: string (nullable = true)
 |-- PM1.0: string (nullable = true)
 |-- PM2.5: string (nullable = true)
 |-- NC0.5: string (nullable = true)
 |-- NC1.0: string (nullable = true)
 |-- NC2.5: string (nullable = true)
 |-- CNT: string (nullable = true)



In [30]:
feature_columns = df.columns
feature_columns.remove('Fire Alarm')
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
rf = RandomForestClassifier(featuresCol='features', labelCol='Fire Alarm', numTrees=100, seed=123)  # You can adjust the number of trees
pipeline = Pipeline(stages=[vector_assembler, rf])
train_data, test_data = df.randomSplit([0.8, 0.2], seed=123)
model = pipeline.fit(train_data)
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="Fire Alarm", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy on test data =", accuracy)


Accuracy on test data = 0.9995226350544991


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 46.0 failed 1 times, most recent failure: Lost task 0.0 in stage 46.0 (TID 77) (Rayen executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext